# Analyze card1-card5 and retreive card id


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# alt.renderers.enable('notebook')

In [5]:
from functions import save_features2, LabelEncoderPopularity

In [6]:
folder_path = './data/'
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
test_transaction['isFraud'] = 'test'
data = pd.concat([train_transaction, test_transaction], axis=0, sort=False )
# data = data.reset_index()
data.set_index('TransactionID',inplace=True)
del train_transaction
del test_transaction

train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')

In [7]:
data_id = pd.concat([train_identity, test_identity], axis=0, sort=False )
# data_id = data.reset_index()
data_id.set_index('TransactionID',inplace=True)
del train_identity
del test_identity


In [8]:
columns = ['isFraud'] + ["card%d"%i for i in range(1,7)]
data = data[columns]


In [9]:
data.head()

,isFraud,card1,card2,card3,card4,card5,card6
TransactionID,,,,,,,
2987000,0,13926,NaN,150.0,discover,142.0,credit
2987001,0,2755,404.0,150.0,mastercard,102.0,credit
2987002,0,4663,490.0,150.0,visa,166.0,debit
2987003,0,18132,567.0,150.0,mastercard,117.0,debit
2987004,0,4497,514.0,150.0,mastercard,102.0,credit


In [10]:
data['card_id'] = data['card1'].map(str)
for col in ['card%d'%i for i in range(2,6)]:
    data['card_id'] += data[col].map(lambda v: " "+str(v))
    
    

In [11]:
L = LabelEncoderPopularity(convert_nan=True)
L.fit(data['card_id'])
data['card_id'] = L.transform(data['card_id'])


In [12]:
data.head()

,isFraud,card1,card2,card3,card4,card5,card6,card_id
TransactionID,,,,,,,,
2987000,0,13926,NaN,150.0,discover,142.0,credit,8346
2987001,0,2755,404.0,150.0,mastercard,102.0,credit,138
2987002,0,4663,490.0,150.0,visa,166.0,debit,100
2987003,0,18132,567.0,150.0,mastercard,117.0,debit,14
2987004,0,4497,514.0,150.0,mastercard,102.0,credit,3122


In [13]:
# save_features2(data,'data', ["card_id"], suffix="card_id")
with pd.HDFStore('./hdf/Store.h5') as store:
    store['initial/card_id'] = data["card_id"]
    print(store.keys())

['/initial/card_id', '/initial/categorial_features', '/initial/encoded', '/initial/numerical_features']


In [32]:
idx = data['card_id'].value_counts().index[3204]
data[data['card_id']  == idx]['isFraud']

TransactionID
3028388       0
3044527       0
3094555       0
3170033       0
3182428       0
3214194       0
3223098       0
3284937       0
3341608       0
3350159       0
3412657       0
3672228    test
3673299    test
3785071    test
3785077    test
3793774    test
3809666    test
3972905    test
3986000    test
4028960    test
4028961    test
4028965    test
4028969    test
4029389    test
4037294    test
4037300    test
4037334    test
4037365    test
4143042    test
Name: isFraud, dtype: object

In [33]:
for i in range (3204,3500):
    idx = data['card_id'].value_counts().index[i]
    s = sum(data[data['card_id']  == idx][data['isFraud']  != 'test']['isFraud'])
    if s >0:
        print (i,s)

3244 1
3263 2
3265 1
3268 1
3274 9
3277 2
3308 1
3314 3
3334 4
3336 3
3341 2
3343 5
3359 3
3367 3
3376 1


KeyboardInterrupt: 

In [36]:
i = 3274
idx = data['card_id'].value_counts().index[i]
tr = data[data['card_id']  == idx][['isFraud']]
tr = pd.merge(tr, data_id, on='TransactionID', how='left')

In [37]:
tr

,isFraud,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
3024990,0,0.0,129273.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,T,T,mobile,iOS Device
3079753,0,0.0,63518.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,firefox 57.0,NaN,NaN,NaN,F,F,T,T,desktop,rv:57.0
3092194,0,-5.0,86713.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,samsung browser 6.2,NaN,NaN,NaN,F,F,T,T,mobile,SAMSUNG
3111296,0,-20.0,85197.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 63.0,24.0,1440x900,match_status:2,T,F,F,T,desktop,MacOS
3111298,0,-20.0,33648.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,...,chrome 63.0,24.0,1440x900,match_status:2,T,F,F,T,desktop,MacOS
3208074,0,-5.0,236471.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,...,edge 16.0,24.0,1600x900,match_status:2,T,F,T,T,desktop,Windows
3216274,0,-5.0,56173.0,NaN,NaN,9.0,-100.0,NaN,NaN,NaN,...,chrome 63.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G950W
3247005,0,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3251342,0,-5.0,120576.0,0.0,0.0,6.0,0.0,NaN,NaN,0.0,...,chrome 61.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN


In [ ]:
tr['id_01'].dropna().hist()

In [ ]:
raise Exception('stop here')

In [ ]:
for i in range(1,6):
    print('i: ', len(train_transaction['card%d'%i].unique()), sum(train_transaction['card%d'%i].isna()))

In [ ]:
# t = train_transaction[train_transaction['card4']=='discover']

In [ ]:
type(train_transaction['card1'].value_counts())

In [ ]:
# plt.figure(figsize=(16,9))
# for i,v in enumerate((1,2,3,5)):
#     plt.subplot(411+i)
#     train_transaction['card%d'%v].hist(bins=1000)
#     t['card%d'%v].hist(bins=1000)

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder

class LabelEncoderPopularity(BaseEstimator, TransformerMixin):
    def __init__(self, convert_nan = False, negate_nan = True):
        self.le = LabelEncoder()
        self.convert_nan = convert_nan
        self.negate_nan = negate_nan
        self.replacement_dict = {}

    def fit(self,x,y=None):
        value_counts = x.value_counts()
        stats = []
        
        for v,c in value_counts.iteritems():
#             print(v,c)
            stats.append((v,c))
        
        if self.convert_nan and not self.negate_nan:
            stats.append ( (np.NaN,sum(x.isna())))
            stats = sorted(stats,key=lambda q: q[1], reverse=True)
        
        for i,s in enumerate(stats):
            self.replacement_dict[s[0]]=i
        
        if self.convert_nan and self.negate_nan:
            self.replacement_dict[np.NaN] = -1
#         print(self.replacement_dict)
#         #Fill missing values with the string 'NaN'
#         a = [q for q in x if q != 'nan']
#         self.le.fit(a)
        return self

    def transform(self,x,y=None):
        tr = x.map(self.replacement_dict)#.astype(np.int32)
        return tr
        
#         #Fill missing values with the string 'NaN'
#         a = [q for q in x if q != 'nan']
#         #Store an ndarray of the current column
#         b = np.array(x)
#         #Replace the elements in the ndarray that are not 'NaN'
#         #using the transformer
#         b[b!='nan'] = self.le.transform(a)
#         b[b=='nan'] = self.nan_value
#         #Overwrite the column in the DataFrame
#         x=b.astype(np.int32)
#         #return the transformed DataFrame
#         return x
# L = LabelEncoderPopularity(convert_nan=True)
# L.fit(train_transaction['card3'])
# t = L.transform(train_transaction['card3'])
# t.head()

In [ ]:
# train_transaction['card3'].astype(np.int32)

In [ ]:
# train_transaction['card3'] = train_transaction['card3'].dropna().map(lambda q: int(q))
# train_transaction['card3']

In [ ]:
# pd.concat([train_transaction['card3'], test_transaction['card3']])

In [ ]:
for col in ['card%d'%i for i in range(1,6)]:
    L = LabelEncoderPopularity(convert_nan=True)
    L.fit(pd.concat([train_transaction[col], test_transaction[col]]))
    train_transaction[col] = L.transform(train_transaction[col])
    test_transaction[col] = L.transform(test_transaction[col])

In [ ]:
train_transaction.head()